# 4.1 Prompt 基础实验

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/04-prompt-engineering/prompt_basics.ipynb)

**预计 API 费用：~$0.01**

本 Notebook 演示如何写出高质量 Prompt，对比不同写法的效果差异。

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("请输入你的 OpenAI API Key: ")

## 实验 1：对比"差 Prompt" vs "好 Prompt"

同样的任务，不同的 Prompt 写法会导致完全不同的输出质量。

In [ ]:
from openai import OpenAI

client = OpenAI()

# 差的 Prompt：模糊、没有具体要求
bad_prompt = "写一个函数计算斐波那契数列"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": bad_prompt}],
)

print("=== 差的 Prompt ===")
print(f"Prompt: {bad_prompt}")
print(f"\nAI 回复：")
print(response.choices[0].message.content)
print(f"\nToken 用量：{response.usage.total_tokens}")

In [ ]:
# 好的 Prompt：具体、有要求、有约束
good_prompt = """
你是一个 Python 算法专家，擅长写高性能代码。

任务：实现斐波那契数列计算函数

要求：
1. 使用动态规划（避免递归）
2. 时间复杂度 O(n)，空间复杂度 O(1)
3. 函数签名：def fibonacci(n: int) -> int
4. 包含参数验证（n >= 0）
5. 添加类型注解和文档字符串
6. 包含 3 个测试用例

输出：完整的 Python 代码，可直接运行。
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": good_prompt}],
)

print("=== 好的 Prompt ===")
print(f"Prompt: {good_prompt}")
print(f"\nAI 回复：")
print(response.choices[0].message.content)
print(f"\nToken 用量：{response.usage.total_tokens}")

**对比结果：**
- 差的 Prompt：可能返回递归实现（性能差）、没有类型注解、没有测试
- 好的 Prompt：返回优化的动态规划实现、完整的类型注解、测试用例

**关键区别：具体的指令 + 明确的约束 + 清晰的输出格式**

## 实验 2：System 消息的魔力

System 消息定义 AI 的角色和风格，对输出影响巨大。

In [ ]:
from openai import OpenAI

client = OpenAI()

question = "解释什么是 API"

# 1. 无 System 消息（默认行为）
response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": question}],
)

print("=== 1. 无 System 消息（默认） ===")
print(response1.choices[0].message.content)
print("\n" + "="*80 + "\n")

In [ ]:
# 2. 技术专家角色
response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "你是一个严谨的计算机科学教授，用学术语言解释概念，包含技术细节和准确定义。"},
        {"role": "user", "content": question},
    ],
)

print("=== 2. System: 技术专家 ===")
print(response2.choices[0].message.content)
print("\n" + "="*80 + "\n")

In [ ]:
# 3. 幽默导师角色
response3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "你是一个幽默的编程导师，喜欢用生活中的类比和比喻解释技术概念，目标受众是完全不懂技术的人。"},
        {"role": "user", "content": question},
    ],
)

print("=== 3. System: 幽默导师 ===")
print(response3.choices[0].message.content)
print("\n" + "="*80 + "\n")

In [ ]:
# 4. 诗人角色（极端测试）
response4 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "你是一个浪漫的诗人，用诗歌的形式解释一切。回复要押韵，充满诗意。"},
        {"role": "user", "content": question},
    ],
)

print("=== 4. System: 诗人 ===")
print(response4.choices[0].message.content)

**关键洞察：**
- System 消息就像"岗位说明书"，定义 AI 的专业领域和回复风格
- 同样的问题，不同的 System 消息会得到完全不同的回答
- 生产环境中，用 System 消息明确 AI 的角色和行为规范

## 实验 3：Prompt 模板实战

为常见任务创建可复用的 Prompt 模板，提高开发效率。

In [ ]:
from openai import OpenAI

client = OpenAI()

# 代码审查模板
def code_review_prompt(code: str, language: str = "Python"):
    return f"""
你是一个资深的 {language} 代码审查专家，擅长发现代码中的问题并提出改进建议。

请审查以下代码，并按这个结构输出：

1. **代码质量评分**（1-10 分）
2. **发现的问题**（按严重程度排序）
   - 每个问题说明：问题 + 影响 + 建议修改
3. **优点**
4. **改进后的代码**（如果有重大问题）

代码：
```{language.lower()}
{code}
```

请给出详细分析。
"""

# 测试代码
code_to_review = '''
def process_users(users):
    result = []
    for user in users:
        if user['age'] > 18:
            result.append(user['name'])
    return result
'''

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": code_review_prompt(code_to_review)}],
)

print("=== 代码审查结果 ===")
print(response.choices[0].message.content)

In [ ]:
# 技术文档生成模板
def doc_generation_prompt(code: str, language: str = "Python"):
    return f"""
你是一个技术文档专家，擅长为代码生成清晰、专业的文档。

请为以下代码生成文档，包括：

1. **功能描述**（一句话概括）
2. **参数说明**（类型、用途）
3. **返回值**（类型、含义）
4. **使用示例**（可运行的代码）
5. **注意事项**（可选）

代码：
```{language.lower()}
{code}
```

输出格式：Markdown
"""

function_code = '''
def calculate_discount(price, discount_rate, min_amount=100):
    if price < min_amount:
        return price
    return price * (1 - discount_rate)
'''

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": doc_generation_prompt(function_code)}],
)

print("=== 生成的文档 ===")
print(response.choices[0].message.content)

## 实验 4：多轮对话中的上下文管理

LLM API 是无状态的，需要手动管理对话历史。

In [ ]:
from openai import OpenAI

client = OpenAI()

# 初始化对话
messages = [
    {"role": "system", "content": "你是一个 Python 编程助手，擅长调试代码和提供优化建议。"}
]

def chat(user_input):
    """发送消息并获取回复"""
    messages.append({"role": "user", "content": user_input})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    
    reply = response.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    
    return reply

# 第一轮对话
print("=== 第 1 轮 ===")
user1 = "这段代码有什么问题？\n```python\nfor i in range(10):\n  print(i)\ntime.sleep(1)\n```"
print(f"用户: {user1}")
print(f"\nAI: {chat(user1)}")
print("\n" + "="*80 + "\n")

In [ ]:
# 第二轮对话（AI 会记住前面的代码）
print("=== 第 2 轮 ===")
user2 = "那我应该怎么导入 time 模块？"
print(f"用户: {user2}")
print(f"\nAI: {chat(user2)}")
print("\n" + "="*80 + "\n")

In [ ]:
# 第三轮对话
print("=== 第 3 轮 ===")
user3 = "有没有更优雅的写法？"
print(f"用户: {user3}")
print(f"\nAI: {chat(user3)}")
print("\n" + "="*80 + "\n")

In [ ]:
# 查看完整的对话历史
print("=== 完整对话历史 ===")
for i, msg in enumerate(messages, 1):
    print(f"{i}. [{msg['role'].upper()}] {msg['content'][:100]}...")
    print()

**关键点：**
- 每次调用 API 都需要传入完整的对话历史
- 对话历史越长，消耗的 token 越多
- 需要在"记忆"和"成本"之间权衡
- 可以定期清理不重要的历史消息

## 动手练习

1. **修改 System 消息**：让 AI 扮演一个 DevOps 专家，解释什么是容器化
2. **创建自己的模板**：为"将需求转换为用户故事"创建一个 Prompt 模板
3. **对比实验**：同一个问题，用不同的 temperature（0, 0.7, 1.5）调用 5 次，观察差异
4. **上下文管理**：实现一个自动截断对话历史的函数（保留最近 10 轮）

## 关键要点总结

1. **System 消息定义角色**：专家 vs 导师 vs 助手，效果完全不同
2. **具体化指令**：不说"写代码"，说"用 FastAPI 实现 CRUD"
3. **提供上下文**：技术栈、约束条件、输出格式
4. **多轮对话需要手动管理历史消息**：API 是无状态的
5. **用模板提高效率**：为常见任务建立可复用的 Prompt 模板

---

**下一步：** 学习 [4.2 进阶 Prompt 技巧](./advanced_techniques.ipynb)，掌握 Few-shot、Chain-of-Thought 等高级方法。